In [2]:
import pandas as pd
import FinanceDataReader as fdr
import statsmodels.api as sm
import seaborn as sns
from datetime import datetime

In [ ]:
# 2022.12.22 펜오션 

In [3]:
STOCK_CODE = '028670'
stock_price = fdr.DataReader(STOCK_CODE, '2022.12','2023.1')

In [4]:
temp_stock_price = stock_price.reset_index()

In [5]:
temp_stock_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    21 non-null     datetime64[ns]
 1   Open    21 non-null     int64         
 2   High    21 non-null     int64         
 3   Low     21 non-null     int64         
 4   Close   21 non-null     int64         
 5   Volume  21 non-null     int64         
 6   Change  21 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(5)
memory usage: 1.3 KB


In [17]:
pre_price = temp_stock_price.query('Date <= 20221222').iloc[-8:-1,:].filter(items =['Date','Close'])
target_price = temp_stock_price.query('Date == 20221222').filter(items = ['Date','Open'])

In [18]:
pre_price = pre_price.rename(columns={'Close': 'Price'})
target_price = target_price.rename(columns={'Open': 'Price'})

In [19]:
df = pd.concat([pre_price,target_price]).reset_index(drop=True)

In [20]:
df['change'] = df['Price'].pct_change()

In [22]:
df.dropna()

,Date,Price,change
1,2022-12-14,5640,-0.001770
2,2022-12-15,5590,-0.008865
3,2022-12-16,5880,0.051878
4,2022-12-19,5880,0.000000
5,2022-12-20,5690,-0.032313
6,2022-12-21,5800,0.019332
7,2022-12-22,5920,0.020690
